## **Actors**

**Actor** is a stateful(remembering the state) worker. <br>
When a new actor is instantiated, a new worker is created.

In [1]:
import ray
ray.init()

2019-03-12 10:52:00,614	WARNING worker.py:1381 -- WARNING: Not updating worker name since `setproctitle` is not installed. Install this with `pip install setproctitle` (or ray[debug]) to enable monitoring of worker processes.
2019-03-12 10:52:00,616	INFO node.py:439 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-03-12_10-52-00_68461/logs.
2019-03-12 10:52:00,723	INFO services.py:364 -- Waiting for redis server at 127.0.0.1:50769 to respond...
2019-03-12 10:52:00,834	INFO services.py:364 -- Waiting for redis server at 127.0.0.1:16026 to respond...
2019-03-12 10:52:00,838	INFO services.py:761 -- Starting Redis shard with 3.44 GB max memory.
2019-03-12 10:52:00,848	INFO services.py:1449 -- Starting the Plasma object store with 5.15 GB memory using /tmp.


{'node_ip_address': None,
 'redis_address': '10.30.198.126:50769',
 'object_store_address': '/tmp/ray/session_2019-03-12_10-52-00_68461/sockets/plasma_store',
 'webui_url': None,
 'raylet_socket_name': '/tmp/ray/session_2019-03-12_10-52-00_68461/sockets/raylet'}

### **Defining and creating an actor**

***ray.remote*** indicates that Counter class will be actors.

In [2]:
@ray.remote
class Counter(object):
    def __init__(self):
        self.value = 0
    
    def increment(self):
        self.value += 1
        return self.value

***Counter.remote()*** can actually create an actor.

In [3]:
a1 = Counter.remote()
a2 = Counter.remote()

### **Using an actor**

In [4]:
a1.increment.remote()
a2.increment.remote()

ObjectID(010000003129c12980aeb228e2a486fe0a6b5694)

When ***a1.increment.remote()*** is called, the following events happens. <br>

1. A task is created.
2. The task is assigned directly to the local scheduler responsible for the actor by the driver’s local scheduler.
3. An object ID is returned.

function in the same object is scheduled on the same local scheduler = ***serial***

In [5]:
# Create ten Counter actors.
counters = [Counter.remote() for _ in range(10)]

# Increment each Counter once and get the results. These tasks all happen in
# parallel.
results = ray.get([c.increment.remote() for c in counters])
print(results)  # prints [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

# Increment the first Counter five times. These tasks are executed serially
# and share state.
results = ray.get([counters[0].increment.remote() for _ in range(5)])
print(results)  # prints [2, 3, 4, 5, 6]

2019-03-12 10:44:51,125	ERROR worker.py:1752 -- WARNING: 12 workers have been started. This could be a result of using a large number of actors, or it could be a consequence of using nested tasks (see https://github.com/ray-project/ray/issues/3644) for some a discussion of workarounds.
2019-03-12 10:44:51,438	ERROR worker.py:1752 -- WARNING: 16 workers have been started. This could be a result of using a large number of actors, or it could be a consequence of using nested tasks (see https://github.com/ray-project/ray/issues/3644) for some a discussion of workarounds.


[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[2, 3, 4, 5, 6]


### **A More Interesting Actor Example - Gym**

In [6]:
import gym

@ray.remote
class GymEnvironment(object):
    def __init__(self, name):
        self.env = gym.make(name)
        self.env.reset()

    def step(self, action):
        return self.env.step(action)

    def reset(self):
        self.env.reset()

In [7]:
pong = GymEnvironment.remote("Pong-v0")

In [8]:
pong.step.remote(0)  # Take action 0 in the simulator.

ObjectID(01000000615a51a0120a6a6f11a4d0c116ff31dd)

This will make it easy usage of parallell simulation in gym environment

### **Actor with Neural Network**

In [2]:
import tensorflow as tf
import ray
import os
from tensorflow.examples.tutorials.mnist import input_data

In [3]:
mnist = input_data.read_data_sets("MNIST_data", one_hot = True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [4]:
@ray.remote
class NeuralNet(object):
    def __init__(self):
        self.sess = tf.Session()
        self.loss = self.build_network()
        self.train_op = self.add_training_op()
        
        self.init_network()
        
    def init_network(self):
        self.sess.run(tf.global_variables_initializer())
    
    def build_network(self):
        self.x = tf.placeholder(tf.float32, [None, 784])
        self.y = tf.placeholder(tf.float32, [None, 10])
        
        self.W = tf.Variable(tf.zeros([784, 10]))
        self.b = tf.Variable(tf.zeros([10]))
    
        score = tf.matmul(self.x, self.W) + self.b
        return tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = score, labels = self.y))
    
    def add_training_op(self):
        return tf.train.GradientDescentOptimizer(0.5).minimize(self.loss)
    
    
    def train(self, data, epoch, batch_size):
        total_batch = data.train.num_examples // batch_size
        for e in range(epoch):
            total_loss = 0.0
            for _ in range(total_batch):
                batch_xs, batch_ys = data.train.next_batch(batch_size)
                _, loss_val = self.sess.run([self.train_op, self.loss], 
                                            feed_dict = {self.x: batch_xs, self.y: batch_ys})
                total_loss += loss_val
            print('Epoch: %04d' %(e+1), end = '  ')
            print('Avg Loss: %.3f' %(total_loss / total_batch))
        

In [7]:
nns = [NeuralNet.remote() for _ in range(2)]

(pid=68600) 2019-03-12 10:53:06.857009: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.1 SSE4.2 AVX AVX2 FMA
(pid=68600) 2019-03-12 10:53:06.857171: I tensorflow/core/common_runtime/process_util.cc:69] Creating new thread pool with default inter op setting: 4. Tune using inter_op_parallelism_threads for best performance.
(pid=68601) 2019-03-12 10:53:06.856735: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.1 SSE4.2 AVX AVX2 FMA
(pid=68601) 2019-03-12 10:53:06.856906: I tensorflow/core/common_runtime/process_util.cc:69] Creating new thread pool with default inter op setting: 4. Tune using inter_op_parallelism_threads for best performance.
(pid=68600) WARNING:tensorflow:From <ipython-input-4-90ce24ab9126>:21: softmax_cross_entropy_with_logits (from tensorflow.python.ops.nn_ops) is deprecated and will 

### ***Multiple Actor is training neural network seperately***

In [10]:
[nn.train.remote(data=mnist, epoch = 10, batch_size = 128) for nn in nns]

[ObjectID(0100000049327a0264ecbc7e0cf2f9eec9ad6496),
 ObjectID(01000000a57452373d13c85b7c0c1ca744247bb2)]

(pid=68601) Epoch: 0001  Avg Loss: 0.266
(pid=68600) Epoch: 0001  Avg Loss: 0.267
(pid=68601) Epoch: 0002  Avg Loss: 0.265
(pid=68600) Epoch: 0002  Avg Loss: 0.265
(pid=68601) Epoch: 0003  Avg Loss: 0.264
(pid=68600) Epoch: 0003  Avg Loss: 0.263
(pid=68601) Epoch: 0004  Avg Loss: 0.262
(pid=68600) Epoch: 0004  Avg Loss: 0.263
(pid=68601) Epoch: 0005  Avg Loss: 0.261
(pid=68600) Epoch: 0005  Avg Loss: 0.261
(pid=68601) Epoch: 0006  Avg Loss: 0.260
(pid=68600) Epoch: 0006  Avg Loss: 0.260
(pid=68601) Epoch: 0007  Avg Loss: 0.260
(pid=68600) Epoch: 0007  Avg Loss: 0.259
(pid=68601) Epoch: 0008  Avg Loss: 0.259
(pid=68600) Epoch: 0008  Avg Loss: 0.258
(pid=68601) Epoch: 0009  Avg Loss: 0.257
(pid=68600) Epoch: 0009  Avg Loss: 0.259
(pid=68601) Epoch: 0010  Avg Loss: 0.257
(pid=68600) Epoch: 0010  Avg Loss: 0.256
